# Load merge data from Data Lake

Get raw data from Data Lake

In [47]:
objectName = 'Invoiced'

StatementMeta(MediumSparkTwo, 1, 36, Finished, Available)

In [48]:
df = (spark
    .read
    .format('csv') 
    .option("delimiter", ";")
    .option("multiline", True)
    .option("quote", "\"")
    .option("escape", "\"")
    .option("header",True)
    .option('path',f'abfss://brbofilesystem@brbodatalake.dfs.core.windows.net/{objectName}')
    .load()
    )

display(df.limit(10))

StatementMeta(MediumSparkTwo, 1, 37, Finished, Available)

SynapseWidget(Synapse.DataFrame, fbf3d5c7-457c-4d6a-bab1-413492fba5a4)

# Handle schema
Remove training header names from dataframe

In [49]:
from pyspark.sql.functions import col

df = df.select([col(c).alias(c[:c.index(' ')]) for c in df.columns])

display(df.limit(10))

StatementMeta(MediumSparkTwo, 1, 38, Finished, Available)

SynapseWidget(Synapse.DataFrame, b2c9e81e-d663-479e-bab7-9f6d73b44eba)

Remove columns that are not used - if they exist

In [50]:
ColumnnamesToDelete = spark.createDataFrame(
    [
        (1, "booking_description"),
        (2, "last_change_date"),
        (3, "color_code_fk"),
        (4, "username"),
        (5, "password"),
        (6, "phone"),
        (7, "email"),
        (8, "sms"),
        (9, "hold_email"),
        (10,"team_sms"),
        (11,"insurance_course_fk"),
        (12,"title"),
        (13,"code")
    ]
    ,["number","columnname"]
    )

for d in ColumnnamesToDelete.collect():
    if d["columnname"] in df.columns:
        df = df.drop(d["columnname"])

StatementMeta(MediumSparkTwo, 1, 39, Finished, Available)

In [51]:
display(df.limit(10))

StatementMeta(MediumSparkTwo, 1, 40, Finished, Available)

SynapseWidget(Synapse.DataFrame, 23829542-765a-4b33-9cbe-c84c9896fa11)

Change column datatypes if they exits

In [52]:
from pyspark.sql.types import StringType,BooleanType,DateType

if 'date' in df.columns:
    df = df.withColumn("date", df["date"].cast(DateType()))

if '_C018' in df.columns:
    df = df.withColumn("_C018", df["_C018"].cast(DateType()))

if 'referral_date' in df.columns:
    df = df.withColumn("referral_date", df["referral_date"].cast(DateType()))


StatementMeta(MediumSparkTwo, 1, 41, Finished, Available)

Remove characters not used in data

In [53]:
from pyspark.sql.functions import regexp_replace

SearchInColumns = spark.createDataFrame(
    [
        (1, "start_time"),
        (2, "end_time")
    ]
    ,["number","columnname"]
    )

CharacterChanges = spark.createDataFrame(
    [
        (1, "\\+", "")
    ]
    ,["number","RegExString","NewString"]
    )

for s in SearchInColumns.collect():
    if s["columnname"] in df.columns:
        for r in CharacterChanges.collect():
            df = df.withColumn(s["columnname"], regexp_replace(col(s["columnname"]), r["RegExString"], r["NewString"]))

display(df.limit(10))

StatementMeta(MediumSparkTwo, 1, 42, Finished, Available)

SynapseWidget(Synapse.DataFrame, b5c0bb9c-ffc2-4ac8-9fab-93a68d68b8f9)

# Write data to Curated area

Write new data to the delta lake in curated version<br>
Notice the APPEND part of the 4th line - this appends all data to the existing data (delta lake)

In [54]:
bronze_loc = f'abfss://brbofilesystem@brbodatalake.dfs.core.windows.net/Curated/{objectName}'

df.write.mode("append").format("delta").save(bronze_loc)


StatementMeta(MediumSparkTwo, 1, 43, Finished, Available)